Introduction:
The following code downloads the Yolo version v8 from the github repository and installs the necessary libraries. The GTSDB dataset in the Yolo format is loaded and the dataset is split into training and test. Finally it is trained on the original dataset and the best model is used to predict the test dataset. The performance for the model is plotted and analysed for further evaluations.

Then the training and test dataset are subjected to noise using gaussian function and it is used to retrain the model again and the best model is used to predict the test dataset. 

Necessary Import statements:

In [ ]:
import torch
import os
import shutil
from IPython.display import Image
from random import choice
from google.colab import drive
import numpy as np
import pandas as pd
import cv2
import random

Dowload the GTSRB training and test dataset:

In [ ]:
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
!unzip GTSRB_Final_Training_Images.zip
!rm GTSRB_Final_Training_Images.zip

!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip
!unzip GTSRB_Final_Test_Images.zip
!rm GTSRB_Final_Test_Images.zip

!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip
!unzip GTSRB_Final_Test_GT.zip
!rm GTSRB_Final_Test_GT.zip
!mv GT-final_test.csv /content/GTSRB/Final_Test/Images/

In [ ]:
%cd ..

In [ ]:
from PIL import Image

INPUT_DIR = '/content/GTSRB'
INPUT_TRAIN_DIR = INPUT_DIR + os.sep + 'Final_Training' + os.sep + 'Images'
INPUT_TEST_DIR = INPUT_DIR + os.sep + 'Final_Test' + os.sep + 'Images'
IN_TEST_GT_DIR = INPUT_DIR + os.sep + 'GTSRB_Final_Test_GT'

OUTPUT_DIR = 'processed_dataset'
OUTPUT_IMAGE_DIR = OUTPUT_DIR + os.sep + 'dataset'
OUTPUT_LABEL_DIR = OUTPUT_DIR + os.sep + 'labels'
OUT_TRAIN_DIR_NAME = 'train_v1'
OUT_VAL_DIR_NAME = 'val_v1'
OUT_TEST_DIR_NAME = 'test_v1'

NUM_PROCESSED_TRAIN_IMAGES = 0
NUM_PROCESSED_TEST_IMAGES = 0

YOLO_CLASSES_MAP = {i: i for i in range(0, 42)}

TRAIN_VALIDATION_SPLIT = 0.2

RANDOM_GENERATOR = np.random.default_rng(12345)


def process_training_class(class_id, in_dir, out_train_image_dir, out_train_label_dir, out_val_image_dir,
                        out_val_label_dir):
    global NUM_PROCESSED_TRAIN_IMAGES

    print(f"Process train data for Class: {class_id}...")
    class_dir_name = str(class_id).zfill(5)

    print(f"Class: {class_dir_name}")

    annotations_df = pd.read_csv(in_dir + os.sep + class_dir_name +
                                 os.sep + f"GT-{class_dir_name}.csv", delimiter=';', header=0)

    for index, row in annotations_df.iterrows():
        file_name = row['Filename']

        random = RANDOM_GENERATOR.random()
        OUTPUT_IMAGE_DIR = out_train_image_dir
        out_label_dir = out_train_label_dir
        if random < TRAIN_VALIDATION_SPLIT:
            out_image_dir = out_val_image_dir
            out_label_dir = out_val_label_dir

        in_file_path = in_dir + os.sep + class_dir_name + os.sep + file_name
        out_file_path = out_image_dir + os.sep + \
                        f"{str(NUM_PROCESSED_TRAIN_IMAGES).zfill(7)}.jpg"

        print(f"Saving output files to path: {out_file_path}")

        img = Image.open(in_file_path)
        img.save(out_file_path)

        out_file_path = out_label_dir + os.sep + \
                        f"{str(NUM_PROCESSED_TRAIN_IMAGES).zfill(7)}.txt"

        bounding_box_x1 = int(row['Roi.X1'])
        bounding_box_y1 = int(row['Roi.Y1'])

        bounding_box_x2 = int(row['Roi.X2'])
        bounding_box_y2 = int(row['Roi.Y2'])

        width = ((bounding_box_x2 - bounding_box_x1) / img.size[0])
        height = ((bounding_box_y2 - bounding_box_y1) / img.size[1])
        x_center = (bounding_box_x1 + (bounding_box_x2 -
                                       bounding_box_x1) / 2) / img.size[0]
        y_center = (bounding_box_y1 + (bounding_box_y2 -
                                       bounding_box_y1) / 2) / img.size[1]

        file = open(out_file_path, 'w')
        file.write(
            f"{YOLO_CLASSES_MAP[class_id]} {x_center} {y_center} {width} {height}")
        file.close()

        NUM_PROCESSED_TRAIN_IMAGES += 1


def process_testing_class(class_index, in_dir, out_image_dir, out_label_dir):
    global NUM_PROCESSED_TEST_IMAGES
    print(f"Process test data for Class: {class_index}...")

    annotations_df = pd.read_csv(
        in_dir + os.sep + 'GT-final_test.csv', delimiter=';', header=0)

    for index, row in annotations_df.iterrows():
        if row['ClassId'] != class_index:
            continue

        file_name = row['Filename']

        in_file_path = in_dir + os.sep + file_name
        out_file_path = out_image_dir + os.sep + \
                        f"{str(NUM_PROCESSED_TEST_IMAGES).zfill(7)}.jpg"

        print(f"Saving to {out_file_path}...")

        img = Image.open(in_file_path)
        img.save(out_file_path)

        out_file_path = out_label_dir + os.sep + \
                        f"{str(NUM_PROCESSED_TEST_IMAGES).zfill(7)}.txt"

        bounding_box_x1 = int(row['Roi.X1'])
        bounding_box_y1 = int(row['Roi.Y1'])

        bounding_box_x2 = int(row['Roi.X2'])
        bounding_box_y2 = int(row['Roi.Y2'])

        width = (bounding_box_x2 - bounding_box_x1) / img.size[0]
        height = (bounding_box_y2 - bounding_box_y1) / img.size[1]
        x_center = (bounding_box_x2 - (bounding_box_x2 - bounding_box_x1) / 2) / img.size[0]
        y_center = (bounding_box_y2 - (bounding_box_y2 - bounding_box_y1) / 2) / img.size[1]

        file = open(out_file_path, 'w')
        file.write(
            f"{YOLO_CLASSES_MAP[class_index]} {x_center} {y_center} {width} {height}")
        file.close()

        NUM_PROCESSED_TEST_IMAGES += 1


def process_class_data(class_index):
    process_training_class(class_index, INPUT_TRAIN_DIR,
                        OUTPUT_IMAGE_DIR + os.sep + OUT_TRAIN_DIR_NAME,
                        OUTPUT_IMAGE_DIR + os.sep + OUT_TRAIN_DIR_NAME,
                        OUTPUT_IMAGE_DIR + os.sep + OUT_VAL_DIR_NAME,
                        OUTPUT_IMAGE_DIR + os.sep + OUT_VAL_DIR_NAME
                        )

    process_testing_class(class_index, INPUT_TEST_DIR,
                       OUTPUT_IMAGE_DIR + os.sep + OUT_TEST_DIR_NAME,
                       OUTPUT_IMAGE_DIR + os.sep + OUT_TEST_DIR_NAME
                       )


def main():
    print("Creating Dataset Directory")

    if os.path.exists(OUTPUT_DIR):
        print(f"Direcotry Already exists! '{OUTPUT_DIR}'")
        print(
            "Delete this directory? y/n [n]")
        if input() != 'y':
            print("Abort")
            return

        shutil.rmtree(OUTPUT_DIR)

    os.mkdir(OUTPUT_DIR)
    os.mkdir(OUTPUT_IMAGE_DIR)

    os.mkdir(OUTPUT_IMAGE_DIR + os.sep + OUT_TRAIN_DIR_NAME)
    os.mkdir(OUTPUT_IMAGE_DIR + os.sep + OUT_VAL_DIR_NAME)
    os.mkdir(OUTPUT_IMAGE_DIR + os.sep + OUT_TEST_DIR_NAME)

    for class_num in range(0, 42):
        process_class_data(class_num)

if __name__ == "__main__":
    main()

Yolo V8:

In [ ]:
%cd content

In [ ]:
#YOLOv8
!git clone https://github.com/ultralytics/ultralytics.git

!mv ultralytics yolov8

%cd yolov8
!pip3 install -qr requirements.txt

In [ ]:
%cd /
%rm -rf /working/yolo/dataset

In [ ]:
images = []
xmls = []

training_path = "/working/yolo/dataset/images/train"
validation_path = "/working/yolo/dataset/images/val"

full_path = "/processed_dataset/dataset/train_v1"

training_ratio = 0.8
validation_ratio = 0.2

total_images = len(os.listdir(full_path))/2

for (dirname, dirs, files) in os.walk(full_path):
    for filename in files:
        if filename.endswith(".txt"):
            xmls.append(filename)
        else:
            images.append(filename)

training_images_count = int(len(images) * training_ratio)
validation_images_count = int(len(images) * validation_ratio)
print("Training images : ",training_images_count)
print("Validation images : ",validation_images_count)

In [ ]:
training_image_path = "/working/yolo/dataset/images/train"
training_label_path = "/working/yolo/dataset/labels/train"
validation_image_path = "/working/yolo/dataset/images/val"
validation_label_path = "/working/yolo/dataset/labels/val"

if not os.path.isdir(training_image_path):
    os.makedirs(training_image_path)
if not os.path.isdir(training_label_path):
    os.makedirs(training_label_path)
if not os.path.isdir(validation_image_path):
    os.makedirs(validation_image_path)
if not os.path.isdir(validation_label_path):
    os.makedirs(validation_label_path)

Yolo V8 - No Noise:

In [ ]:
for x in range(training_images_count):
    file_jpg = choice(images)
    file_xml = file_jpg[:-4] + ".txt"

    shutil.copy(os.path.join(full_path,file_jpg),os.path.join(training_image_path,file_jpg))
    shutil.copy(os.path.join(full_path,file_xml),os.path.join(training_label_path,file_xml))

    images.remove(file_jpg)
    xmls.remove(file_xml)

In [ ]:
for x in range(validation_images_count):
    file_jpg = choice(images)
    file_xml = file_jpg[:-4] + ".txt"

    shutil.copy(os.path.join(full_path,file_jpg),os.path.join(validation_image_path,file_jpg))
    shutil.copy(os.path.join(full_path,file_xml),os.path.join(validation_label_path,file_xml))

    images.remove(file_jpg)
    xmls.remove(file_xml)

In [ ]:
%%writefile /content/yolov8/data/traffic_sign_data.yaml

path: /working/yolo/dataset
train: images/train/
val: images/val/

nc: 43

names: ['Speed limit (20km/h)',
'Speed limit (30km/h)',
'Speed limit (50km/h)',
'Speed limit (60km/h)',
'Speed limit (70km/h)',
'Speed limit (80km/h)',
'End of speed limit (80km/h)',
'Speed limit (100km/h)',
'Speed limit (120km/h)',
'No passing',
'No passing veh over 3.5 tons',
'Right-of-way at intersection',
'Priority road',
'Yield',
'Stop',
'No vehicles',
'Veh > 3.5 tons prohibited',
'No entry',
'General caution',
'Dangerous curve left',
'Dangerous curve right',
'Double curve',
'Bumpy road',
'Slippery road',
'Road narrows on the right',
'Road work',
'Traffic signals',
'Pedestrians',
'Children crossing',
'Bicycles crossing',
'Beware of ice/snow',
'Wild animals crossing',
'End speed + passing limits',
'Turn right ahead',
'Turn left ahead',
'Ahead only',
'Go straight or right',
'Go straight or left',
'Keep right',
'Keep left',
'Roundabout mandatory',
'End of no passing',
'End no passing veh > 3.5 tons']

In [ ]:
%cd /content/yolov8

In [ ]:
!pip3 install ultralytics

In [ ]:
!yolo task=detect mode=train model=yolov8m.pt imgsz=128 data=/content/yolov8/data/traffic_sign_data.yaml epochs=15 batch=8 name=yolov8m_v8_50e

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/detect/yolov8m_v8_50e

Yolo V8 - Noise:

In [ ]:
%cd /content
%rm -rf yolov8

%cd /
%rm -rf /working/yolo/dataset

In [ ]:
%cd /content
#YOLOv8
!git clone https://github.com/ultralytics/ultralytics.git

!mv ultralytics yolov8

# Change the current working directory.
%cd yolov8

!pip install -qr requirements.txt

In [ ]:
%cd /

In [ ]:
images = []
xmls = []

training_path = "/working/yolo/dataset/images/train"
validation_path = "/working/yolo/dataset/images/val"

full_path = "/processed_dataset/dataset/train_v1"

training_ratio = 0.8
validation_ratio = 0.2

total_images = len(os.listdir(full_path))/2

for (dirname, dirs, files) in os.walk(full_path):
    for filename in files:
        if filename.endswith(".txt"):
            xmls.append(filename)
        else:
            images.append(filename)

# counting range for cycles
training_images_count = int(len(images) * training_ratio)
validation_images_count = int(len(images) * validation_ratio)
print("Training images are : ",training_images_count)
print("Validation images are : ",validation_images_count)

In [ ]:
training_image_path = "/working/yolo/dataset/images/train"
training_label_path = "/working/yolo/dataset/labels/train"
validation_image_path = "/working/yolo/dataset/images/val"
validation_label_path = "/working/yolo/dataset/labels/val"

if not os.path.isdir(training_image_path):
    os.makedirs(training_image_path)
if not os.path.isdir(training_label_path):
    os.makedirs(training_label_path)
if not os.path.isdir(validation_image_path):
    os.makedirs(validation_image_path)
if not os.path.isdir(validation_label_path):
    os.makedirs(validation_label_path)

In [ ]:
%cd /
%mkdir /temp

In [ ]:
from PIL import Image

In [ ]:
for x in range(training_images_count):
    file_jpg = choice(images)
    print(file_jpg)
    file_xml = file_jpg[:-4] + ".txt"

    image_path= os.path.join(full_path, file_jpg)
    print(image_path)
    image = Image.open(image_path)
    image_np = np.array(image)
    mean=0
    st=0.3
    gauss = np.random.normal(mean, st, image_np.shape)
    gauss = (gauss * 255).astype('uint8')
    noisy_image = cv2.add(image_np, gauss)
    noisy_image_pil = Image.fromarray(noisy_image)
    temp_noisy_image_path = os.path.join("/temp"+file_jpg)
    noisy_image_pil.save(temp_noisy_image_path)

    shutil.copy(temp_noisy_image_path,os.path.join(training_image_path,file_jpg))
    shutil.copy(os.path.join(full_path,file_xml),os.path.join(training_label_path,file_xml)
    os.remove(temp_noisy_image_path)
    images.remove(file_jpg)
    xmls.remove(file_xml)

In [ ]:
for x in range(validation_images_count):
    file_jpg = choice(images)
    file_xml = file_jpg[:-4] + ".txt"

    image_path= os.path.join(full_path, file_jpg)
    print(image_path)
    image = Image.open(image_path)
    image_np = np.array(image)
    mean=0
    st=0.3
    gauss = np.random.normal(mean, st, image_np.shape)
    gauss = (gauss * 255).astype('uint8')
    noisy_image = cv2.add(image_np, gauss)
    noisy_image_pil = Image.fromarray(noisy_image)
    temp_noisy_image_path = os.path.join("/temp"+file_jpg)
    noisy_image_pil.save(temp_noisy_image_path)

    shutil.copy(temp_noisy_image_path,os.path.join(validation_image_path,file_jpg))
    shutil.copy(os.path.join(full_path,file_xml),os.path.join(validation_label_path,file_xml))
    os.remove(temp_noisy_image_path)

    images.remove(file_jpg)
    xmls.remove(file_xml)

In [ ]:
cd /content/yolov8

In [ ]:
!pip3 install ultralytics

In [ ]:
!yolo task=detect mode=train model=yolov8m.pt imgsz=128 data=/content/yolov8/data/traffic_sign_data.yaml epochs=15 batch=8 name=yolov8m_v8_50e